In [79]:
import graphlab

# Load the CIFAR-10 dataset

In [103]:
image_train = graphlab.SFrame('image_train_data/')
image_test = graphlab.SFrame('image_test_data/')

In [4]:
#reminder
image_train.head()

id,image,label,deep_features,image_array
24,Height: 32 Width: 32,bird,"[0.242871761322,1.09545373917, 0.0, ...","[73.0, 77.0, 58.0, 71.0,68.0, 50.0, 77.0, 69.0, ..."
33,Height: 32 Width: 32,cat,"[0.525087952614, 0.0,0.0, 0.0, 0.0, 0.0, ...","[7.0, 5.0, 8.0, 7.0, 5.0,8.0, 5.0, 4.0, 6.0, 7.0, ..."
36,Height: 32 Width: 32,cat,"[0.566015958786, 0.0,0.0, 0.0, 0.0, 0.0, ...","[169.0, 122.0, 65.0,131.0, 108.0, 75.0, ..."
70,Height: 32 Width: 32,dog,"[1.12979578972, 0.0, 0.0,0.778194487095, 0.0, ...","[154.0, 179.0, 152.0,159.0, 183.0, 157.0, ..."
90,Height: 32 Width: 32,bird,"[1.71786928177, 0.0, 0.0,0.0, 0.0, 0.0, ...","[216.0, 195.0, 180.0,201.0, 178.0, 160.0, ..."
97,Height: 32 Width: 32,automobile,"[1.57818555832, 0.0, 0.0,0.0, 0.0, 0.0, ...","[33.0, 44.0, 27.0, 29.0,44.0, 31.0, 32.0, 45.0, ..."
107,Height: 32 Width: 32,dog,"[0.0, 0.0,0.220677852631, 0.0, ...","[97.0, 51.0, 31.0, 104.0,58.0, 38.0, 107.0, 61.0, ..."
121,Height: 32 Width: 32,bird,"[0.0, 0.23753464222, 0.0,0.0, 0.0, 0.0, ...","[93.0, 96.0, 88.0, 102.0,106.0, 97.0, 117.0, ..."
136,Height: 32 Width: 32,automobile,"[0.0, 0.0, 0.0, 0.0, 0.0,0.0, 7.5737862587, 0.0, ...","[35.0, 59.0, 53.0, 36.0,56.0, 56.0, 42.0, 62.0, ..."
138,Height: 32 Width: 32,bird,"[0.658935725689, 0.0,0.0, 0.0, 0.0, 0.0, ...","[205.0, 193.0, 195.0,200.0, 187.0, 193.0, ..."


# Train a nearest-neighbors model for retrieving images using deep features

In [5]:
knn_model = graphlab.nearest_neighbors.create(image_train,
                                             features=['deep_features'],
                                             label='id')

Starting brute force nearest neighbors model training.

# Use image retrieval model with deep features to find similar images

In [6]:
graphlab.canvas.set_target('ipynb')
cat = image_train[18:19]
cat['image'].show()

In [7]:
## find nearest neighbors of this cat
knn_model.query(cat)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 12.705ms     |

| Done         |         | 100         | 164.283ms    |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,384,0.0,1
0,6910,36.9403137951,2
0,39777,38.4634888975,3
0,36870,39.7559623119,4
0,41734,39.7866014148,5


In [8]:
## nearest neighbors are image in reference_label 
def get_images_from_ids(query_result):
    # return images of id in reference_label
    return image_train.filter_by(query_result['reference_label'],
                                'id')

In [9]:
cat_neighbors = get_images_from_ids(knn_model.query(cat))

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 8.599ms      |

| Done         |         | 100         | 134.251ms    |

+--------------+---------+-------------+--------------+

In [10]:
cat_neighbors['image'].show()

In [11]:
car = image_train[8:9]
car['image'].show()

In [12]:
get_images_from_ids(knn_model.query(car))['image'].show()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 15.034ms     |

| Done         |         | 100         | 142.91ms     |

+--------------+---------+-------------+--------------+

# Let's create a lambda to find and show nearest neighbor images

In [13]:
show_neighbors = lambda i: get_images_from_ids(knn_model.query(image_train[i:i+1]))['image'].show()

In [14]:
show_neighbors(8)

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.0498753   | 9.93ms       |

| Done         |         | 100         | 130.407ms    |

+--------------+---------+-------------+--------------+

## Begin Quiz --------

In [27]:
trainLabel = image_train['label']
trainLabel.sketch_summary()


+------------------+-------+----------+
|       item       | value | is exact |
+------------------+-------+----------+
|      Length      |  2005 |   Yes    |
| # Missing Values |   0   |   Yes    |
| # unique values  |   4   |    No    |
+------------------+-------+----------+

Most frequent items:
+-------+------------+-----+-----+------+
| value | automobile | cat | dog | bird |
+-------+------------+-----+-----+------+
| count |    509     | 509 | 509 | 478  |
+-------+------------+-----+-----+------+


In [32]:
image_train['label'].unique()

dtype: str
Rows: 4
['bird', 'dog', 'cat', 'automobile']

In [34]:
birdFrame = image_train[image_train['label'] == 'bird']
dogFrame = image_train[image_train['label'] == 'dog']
catFrame = image_train[image_train['label'] == 'cat']
carFrame = image_train[image_train['label'] == 'automobile']

In [44]:
catFrame['image'].show()

In [46]:
bird_model = graphlab.nearest_neighbors.create(birdFrame,
                                             features=['deep_features'],
                                             label='id')
dog_model = graphlab.nearest_neighbors.create(dogFrame,
                                             features=['deep_features'],
                                             label='id')
cat_model = graphlab.nearest_neighbors.create(catFrame,
                                             features=['deep_features'],
                                             label='id')
car_model = graphlab.nearest_neighbors.create(carFrame,
                                             features=['deep_features'],
                                             label='id')

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

Starting brute force nearest neighbors model training.

In [50]:
image_test[0:1]['image'].show()

In [53]:
## find nearest neighbors of this cat
cat_model.query(image_test[0:1])

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 10.446ms     |

| Done         |         | 100         | 64.954ms     |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,16289,34.623719208,1
0,45646,36.0068799284,2
0,32139,36.5200813436,3
0,25713,36.7548502521,4
0,331,36.8731228168,5


In [54]:
get_images_from_ids(cat_model.query(image_test[0:1]))['image'].show()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 7.726ms      |

| Done         |         | 100         | 59.499ms     |

+--------------+---------+-------------+--------------+

In [55]:
dog_model.query(image_test[0:1])

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 8.713ms      |

| Done         |         | 100         | 56.406ms     |

+--------------+---------+-------------+--------------+

query_label,reference_label,distance,rank
0,16976,37.4642628784,1
0,13387,37.5666832169,2
0,35867,37.6047267079,3
0,44603,37.7065585153,4
0,6094,38.5113254907,5


In [56]:
get_images_from_ids(dog_model.query(image_test[0:1]))['image'].show()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 7.629ms      |

| Done         |         | 100         | 54.961ms     |

+--------------+---------+-------------+--------------+

## Quiz Question 3

In [65]:
catMean = cat_model.query(image_test[0:1])['distance'].mean()
dogMean = dog_model.query(image_test[0:1])['distance'].mean()

Starting pairwise querying.

+--------------+---------+-------------+--------------+

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 8.066ms      |

...

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 0            | 1       | 0.196464    | 11.309ms     |

| Done         |         | 100         | 56.941ms     |

+--------------+---------+-------------+--------------+

In [67]:
print "cat-mean distance = ", catMean
print "dog-mean distance = ", dogMean
print "Average distance to a cat is smaller"

cat-mean distance =  36.1557307098
dog-mean distance =  37.7707113618
Average distance to a cat is smaller


## challengin question 

In [104]:
image_test_bird = image_test[image_test['label'] == 'bird']
image_test_dog = image_test[image_test['label'] == 'dog']
image_test_cat = image_test[image_test['label'] == 'cat']
image_test_car = image_test[image_test['label'] == 'automobile']

### finding closest neighbor in to the dog test data 

In [105]:
dog_cat_neighbors = cat_model.query(image_test_dog,k=1)
dog_dog_neighbors = dog_model.query(image_test_dog,k=1)
dog_car_neighbors = car_model.query(image_test_dog,k=1)
dog_bird_neighbors = bird_model.query(image_test_dog,k=1)

Starting blockwise querying.

max rows per data block: 4348

number of reference data blocks: 8

number of query data blocks: 1

+--------------+---------+-------------+--------------+

...

| Query points | # Pairs | % Complete. | Elapsed Time |

+--------------+---------+-------------+--------------+

| 1000         | 59000   | 12.3431     | 208.275ms    |

| Done         | 478000  | 100         | 295.731ms    |

+--------------+---------+-------------+--------------+

In [106]:
dog_distance = graphlab.SFrame({'dog-dog': dog_dog_neighbors['distance'],
                                'dog-cat': dog_cat_neighbors['distance'],
                                'dog-car': dog_car_neighbors['distance'],
                                'dog-bird': dog_bird_neighbors['distance']})

In [107]:
dog_distance

dog-bird,dog-car,dog-cat,dog-dog
41.7538647304,41.9579761457,36.4196077068,33.4773590373
41.3382958925,46.0021331807,38.8353268874,32.8458495684
38.6157590853,42.9462290692,36.9763410854,35.0397073189
37.0892269954,41.6866060048,34.5750072914,33.9010327697
38.272288694,39.2269664935,34.778824791,37.4849250909
39.1462089236,40.5845117698,35.1171578292,34.945165344
40.523040106,45.1067352961,40.6095830913,39.0957278345
38.1947918393,41.3221140974,39.9036867306,37.7696131032
40.1567131661,41.8244654995,38.0674700168,35.1089144603
45.5597962603,45.4976929401,42.7258732951,43.2422832585


In [108]:
def is_dog_correct(row):
    test1 = (row['dog-dog']<row['dog-cat'])
    test2 = (row['dog-dog']<row['dog-car'])
    test3 = (row['dog-dog']<row['dog-bird'])
    if (test1 and test2 and test3):
        return 1;
    else:
        return 0;

total = 0;
for row in dog_distance:
    out = is_dog_correct(row)
    total = total + out;
print total
    

678


In [109]:
dog_distance.apply(is_dog_correct).sum()

678

In [112]:
len(image_test_dog)

1000

In [113]:
## 678/1000 ~ 0.678 = 67.8% accuracy